In [ ]:
import numpy as np
import random
import copy
from multiprocessing import Pool, cpu_count
import matplotlib.pyplot as plt
import timeit

class Piece():
    def __init__(self, skirt, heights):
        self.skirt = np.array(skirt)
        self.heights = np.array(heights)
        self.width = self.heights.size

class Board():
    def __init__(self):
        self.width = 10
        self.height = 20
        self.state = np.zeros((self.height, self.width), dtype=int)
        self.tops = np.zeros(self.width, dtype=int)
        self.maxHeight = max(self.tops)
        self.linesCleared = 0
        self.totalLinesCleared = 0
        self.fitness = 0
        self.weights = np.zeros(7)
        self.features = np.zeros(7)
        self.score = 0
        self.setPieces()

    def printState(self):
        for i in self.state[::-1]:
            for j in i:
                print(j, end=' ')
            print()
        print()

    def drop(self, piece, xOffset):
        # returns true if succesful drop
        temp = []
        for i in range(piece.width):
            temp.append(self.tops[xOffset + i] - piece.skirt[i])

        tempMax = max(temp)
        for i in range(len(temp)):
            temp[i] = tempMax - temp[i]

        for i in range(len(temp)):
            temp[i] += self.tops[xOffset + i]

        for j in range(piece.width):
            for i in range(piece.heights[j]):
                if i + temp[j] >= self.height:
                    return False
                self.state[i + temp[j]][j + xOffset] = 1
            self.tops[xOffset + j] = temp[j] + piece.heights[j]

        # update maxHeight
        self.maxHeight = max(self.tops)

        # check for cleared lines
        self.clearLines()

        return True

    def clearLines(self):
        # checks for full lines, returns number of lines cleared
        count = 0
        i = 0

        while i < self.maxHeight:
            # check if lines are full
            if 0 not in self.state[i]:
                # move lines down
                self.state[i:-1] = self.state[i + 1:]
                self.state[-1] = np.zeros(self.width)
                # keep track of lines cleared
                count += 1
                # update other states
                self.maxHeight -= 1
                self.tops -= 1
            else:
                i += 1
        self.totalLinesCleared += count
        self.linesCleared = count

        scores = {0: 0, 1: 100, 2: 300, 3: 500, 4: 3200}

        self.fitness += scores[count]

    def setPieces(self):
        iPiece1 = Piece([0], [4])
        iPiece2 = Piece([0, 0, 0, 0], [1, 1, 1, 1])
        jPiece1 = Piece([0, 0, 0], [2, 1, 1])
        jPiece2 = Piece([0, 2], [3, 1])
        jPiece3 = Piece([1, 1, 0], [1, 1, 2])
        jPiece4 = Piece([0, 0], [1, 3])
        lPiece1 = Piece([0, 0, 0], [1, 1, 2])
        lPiece2 = Piece([0, 0], [3, 1])
        lPiece3 = Piece([0, 1, 1], [2, 1, 1])
        lPiece4 = Piece([2, 0], [1, 3])
        oPiece1 = Piece([0, 0], [2, 2])
        sPiece1 = Piece([0, 0, 1], [1, 2, 1])
        sPiece2 = Piece([1, 0], [2, 2])
        tPiece1 = Piece([0, 0, 0], [1, 2, 1])
        tPiece2 = Piece([0, 1], [3, 1])
        tPiece3 = Piece([1, 0, 1], [1, 2, 1])
        tPiece4 = Piece([1, 0], [1, 3])
        zPiece1 = Piece([1, 0, 0], [1, 2, 1])
        zPiece2 = Piece([0, 1], [2, 2])

        iPieces = [iPiece1, iPiece2]
        jPieces = [jPiece1, jPiece2, jPiece3, jPiece4]
        lPieces = [lPiece1, lPiece2, lPiece3, lPiece4]
        oPieces = [oPiece1]
        sPieces = [sPiece1, sPiece2]
        tPieces = [tPiece1, tPiece2, tPiece3, tPiece4]
        zPieces = [zPiece1, zPiece2]

        # self.pieces -> piece -> orientation of piece
        self.pieces = [iPieces, jPieces, lPieces, oPieces, sPieces, tPieces, zPieces]

    def setWeights(self, weights):
        # assign weights passed in
        self.weights = weights

    def calcFeatures(self):
        # calculates the features:

        # number of holes and depth of wholes
        holes = 0
        holeDepths = 0
        # iterate through each col
        for i in range(self.width):
            curr = self.state[:self.tops[i], i]
            # find the zeros
            zeros = np.where(curr == 0)[0]
            # add the number of zeros
            holes += zeros.size
            # add the depth of the zeros, calculated by the self.tops[i] - index
            holeDepths += np.sum(self.tops[i] - zeros)

        # number of blocks(non zero) in column 0
        col0 = np.count_nonzero(self.state[:self.tops[0], 0])

        # bumpiness of the tops, sum and abs
        diff = np.diff(self.tops)
        bumps = np.sum(np.abs(diff))

        # max height
        maxHeight = self.maxHeight

        # lines cleared from this drop
        linesCleared = self.linesCleared

        # if tetris
        tetris = 1 if linesCleared == 4 else 0

        if tetris == 1:
            print("slam")

        self.features =  [holes, holeDepths, col0, bumps, maxHeight, linesCleared, tetris]

    def calcScore(self):
        # calculates the score based off the weights and features
        self.score = np.dot(self.weights, self.features)
        # print(self.score, self.features)

    def nextState(self):
        # finds the best next state and updates itself to it
        # returns False if game is over
        bestScore = -np.inf
        # select a random piece
        temp = random.randint(0, len(self.pieces) - 1)
        nextPiece = self.pieces[temp]
        # print(temp)

        alive = False

        for orientation in nextPiece:
            for offset in range(self.width - orientation.width + 1):
                # for each possible position, place and append to states
                nextBoard = copy.deepcopy(self)
                if not nextBoard.drop(orientation, offset):
                    continue
                alive = True
                nextBoard.calcFeatures()
                nextBoard.calcScore()
                if nextBoard.score > bestScore:
                    bestScore = nextBoard.score
                    bestState = nextBoard

        if alive:
            self.__dict__ = copy.deepcopy(bestState.__dict__)
        return alive


In [343]:
class Game():
    def __init__(self, weights):
        self.weights = weights
        board = Board()
        board.setWeights(weights)

        while board.nextState():
            board.printState()


# game = Game([[-0.79896526, -0.59458776, -0.65906788, -0.38404566, 0.08857257, 0.22459613, 0.51050845]])
# game = Game([-1., -0.25771718, 0.24803469, -0.2745283, 0.05661302, 0.13177877, 0.8236142])
# game  = Game([-0.89895443, -0.01003559, 0.19795485, -0.13720102, -0.12410522, 0.13771056, -0.67180279])

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 1 1 1 

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 0 1 1 
0 0 0 0 0 0 0 0 1 1 
0 0 0 0 0 0 0 1 1 1 

0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0

In [ ]:
random.randint(0, 6)